### Session set up

Import required modules

In [1]:
import pandas as pd
import re

### Read in and format data

Read in data from [Brisbane City Council](https://www.brisbane.qld.gov.au/clean-and-green/rubbish-tips-and-bins/rubbish-collections/kerbside-large-item-collection-service) (kerbside collection 2022-23, data scraped from website), [QGSO](https://www.qgso.qld.gov.au/geographies-maps/concordances/place-names-concordance) (Place Names Concordance 2020, used under CC-BY), and [ABS](https://www.abs.gov.au/AUSSTATS/abs@.nsf/Lookup/2033.0.55.001Main+Features12016?OpenDocument) (SEIFA SA2 Indexes, used under CC-BY). Note SEIFA from 2021 census not yet released. Index of Relative Socio-economic Advantage and Disadvantage used as interested in areas that have high access to material and social resources.

In [2]:
kerbside = pd.read_csv("../data/kerbside collection 2022-23.csv", usecols=["Suburbs", "Collection_Date"])
place_concord = pd.read_excel("../data/Place Names Concordance 2020.xlsx", sheet_name = "2020")
seifa_16 = pd.read_excel("../data/2033055001 - sa2 indexes.xls", sheet_name = "Table 3", skiprows = 4)

Check data read in as expected 

In [3]:
kerbside.head()

,Suburbs,Collection_Date
0,Acacia Ridge,29-Aug-22
1,Algester,11-Jul-22
2,Calamvale,11-Jul-22
3,Parkinson,11-Jul-22
4,Auchenflower,18-Jul-22


In [4]:
place_concord.head()

,Place name (2020),Alternative place name (2020),Place type (2020),Place name longitude (GDA94),Place name latitude (GDA94),Suburb (2020),Postcode (2019),LGA code (2020),LGA name (2020),SA1 code (2016),...,SA2 name (2011),SA3 code (2011),SA3 name (2011),SA4 code (2011),SA4 name (2011),GCCSA code (2011),GCCSA name (2011),SED name (2017),SED name (2008),CED name (2019)
0,A Creek,NaN,STRM,151.200000,-25.500000,Coonambula,4626,35760,North Burnett (R),31902150304,...,Gayndah - Mundubbera,31902,Burnett,319,Wide Bay,3RQLD,Rest of Qld,Callide,Callide,Flynn
1,A Creek,NaN,STRM,151.584442,-24.648333,Gindoran,4676,33360,Gladstone (R),30805153006,...,Agnes Water - Miriam Vale,30802,Gladstone - Biloela,308,Fitzroy,3RQLD,Rest of Qld,Burnett,Burnett,Flynn
2,A Flat Creek,NaN,STRM,152.317146,-26.374427,Manumbar,4601,33620,Gympie (R),31903151506,...,Kilkivan,31903,Gympie - Cooloola,319,Wide Bay,3RQLD,Rest of Qld,Nanango,Callide,Wide Bay
3,A W Creek,NaN,STRM,142.949720,-13.003610,Lockhart River,4892,34570,Lockhart River (S),31501139615,...,Cape York,31501,Far North,315,Queensland - Outback,3RQLD,Rest of Qld,Cook,Cook,Leichhardt
4,Aarons Folly Gully,NaN,STRM,148.666670,-21.800000,Strathfield,4742,33980,Isaac (R),31201133807,...,Broadsound - Nebo,31201,Bowen Basin - North,312,Mackay,3RQLD,Rest of Qld,Burdekin,Mirani,Capricornia


Update names that will be on output

In [5]:
place_concord.rename(columns={"Place name (2020)": "Suburbs"
                             , "SA2 code (2016)": "sa2_16_code"}
                    , inplace=True)

In [6]:
seifa_16.head()

,2016 Statistical Area Level 2 (SA2) 9-Digit Code,2016 Statistical Area Level 2 (SA2) Name,Usual Resident Population,Unnamed: 3,Unnamed: 4,Ranking within Australia,Unnamed: 6,Unnamed: 7,Unnamed: 8,Ranking within State or Territory,Unnamed: 10,Unnamed: 11,Unnamed: 12,Minimum score for SA1s in area,Maximum score for SA1s in area,% Usual Resident Population without an SA1 level score
0,NaN,NaN,NaN,Score,NaN,Rank,Decile,Percentile,NaN,State,Rank,Decile,Percentile,NaN,NaN,NaN
1,101021007,Braidwood,3872.0,997,NaN,1092,5,50,NaN,NSW,295,6,53,936.0,1063.0,0.000000
2,101021008,Karabar,8247.0,985,NaN,979,5,45,NaN,NSW,264,5,48,710.0,1125.0,0.000000
3,101021009,Queanbeyan,10842.0,982,NaN,943,5,44,NaN,NSW,258,5,47,858.0,1077.0,0.001568
4,101021010,Queanbeyan - East,4786.0,1013,NaN,1246,6,58,NaN,NSW,325,6,59,926.0,1163.0,0.005224


Make required updates to data for easier referencing. Will only rename columns to be kept in code.

In [7]:
seifa_16.dropna(1, "all", inplace=True)
seifa_16.rename(
    columns={"2016 Statistical Area Level 2  (SA2) 9-Digit Code": "sa2_16_code"
            , "2016 Statistical Area Level 2 (SA2) Name ": "sa2_16_name"
            , "Ranking within State or Territory": "state"
            , "Unnamed: 10": "state_rank"
            , "Unnamed: 11": "state_decile"
            , "Unnamed: 12": "state_percentile"
            , "Minimum score for SA1s in area": "min_sa1_score"
            , "Maximum score for SA1s in area": "max_sa1_score"}
    , inplace=True
)

Confirm through visual inspection that the desired columns and names have been updated.

In [8]:
seifa_16.tail()

,sa2_16_code,sa2_16_name,Usual Resident Population,Unnamed: 3,Ranking within Australia,Unnamed: 6,Unnamed: 7,state,state_rank,state_decile,state_percentile,min_sa1_score,max_sa1_score,% Usual Resident Population without an SA1 level score
2182,901021002,Cocos (Keeling) Islands,544.0,890,194,1,9,OT,NaN,NaN,NaN,830.0,1061.0,0.000000
2183,901031003,Jervis Bay,391.0,863,97,1,5,OT,NaN,NaN,NaN,709.0,1019.0,0.023018
2184,901041004,Norfolk Island,1748.0,975,877,5,41,OT,NaN,NaN,NaN,927.0,1020.0,0.000000
2185,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2186,© Commonwealth of Australia 2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Reduce SEIFA information to just QLD for more reliable matching as there are duplicate localities within and outside QLD. Also confirm that data filtered correctly by checking that the first digit of SA2 is 3 (ABS state code for QLD). 

In [9]:
seifa_16 = seifa_16[seifa_16["state"] == "QLD"]
all((seifa_16["sa2_16_code"] >= 300000000) & (seifa_16["sa2_16_code"] < 400000000))

True

Reduce place name concordance to just Brisbane localities. The SA2 code on the file is tied to the place name column. Place name will be used to add on the SA2 code for the scraped Kerbside list.

In [10]:
place_concord = place_concord[place_concord["LGA code (2020)"] == 31000]

### Clean data

Identify if there are any localities that don't share the same name. Want to match kerbside suburbs against the SEIFA deciles based on locality name.

In [11]:
locality_check = kerbside["Suburbs"].isin(place_concord["Suburbs"])
kerbside[~locality_check]

,Suburbs,Collection_Date
10,Chuwar,25-Jul-22
15,Mt Crosby,25-Jul-22
29,Mt Ommaney,15-Aug-22
44,Macgregor,29-Aug-22
70,Albion,24-Oct-22
77,Ascot,31-Oct-22
108,The Gap,23-Jan-23
121,Red Hill,13-Feb-23
149,West End,17-Apr-23
169,Mt Gravatt,29-May-23


Identify why these names don't match to the list of Kerbside collection suburbs

In [12]:
missing_localities = place_concord["Suburbs"].str.contains("chuwar|crosby|ommaney|macgregor|albion|ascot|gap|red hill|west end|gravatt|mackenzie", flags=re.IGNORECASE)
place_concord[missing_localities]

,Suburbs,Alternative place name (2020),Place type (2020),Place name longitude (GDA94),Place name latitude (GDA94),Suburb (2020),Postcode (2019),LGA code (2020),LGA name (2020),SA1 code (2016),...,SA2 name (2011),SA3 code (2011),SA3 name (2011),SA4 code (2011),SA4 name (2011),GCCSA code (2011),GCCSA name (2011),SED name (2017),SED name (2008),CED name (2019)
212,Albion (Brisbane City),NaN,SUB,153.044170,-27.433610,Albion,4010,31000,Brisbane (C),30503111913,...,Albion,30503,Brisbane Inner - North,305,Brisbane Inner City,3GBRI,Greater Brisbane,Clayfield,Clayfield,Brisbane
790,Ascot (Brisbane City),NaN,SUB,153.063890,-27.429720,Ascot,4007,31000,Brisbane (C),30503112112,...,Ascot,30503,Brisbane Inner - North,305,Brisbane Inner City,3GBRI,Greater Brisbane,Clayfield,Clayfield,Brisbane
6664,Chuwar (Brisbane City),NaN,LOCB,152.772766,-27.549166,Chuwar,4306,31000,Brisbane (C),31003128811,...,Ipswich - North,31003,Ipswich Inner,310,Ipswich,3GBRI,Greater Brisbane,Ipswich West,Ipswich West,Blair
12278,Gap Creek,NaN,STRM,152.915584,-27.497826,Brookfield,4069,31000,Brisbane (C),30402108714,...,Brookfield - Kenmore Hills,30402,Kenmore - Brookfield - Moggill,304,Brisbane - West,3GBRI,Greater Brisbane,Moggill,Moggill,Ryan
18754,MacGregor,NaN,SUB,153.075830,-27.565000,MacGregor,4109,31000,Brisbane (C),30303106110,...,Macgregor (Qld),30303,Mt Gravatt,303,Brisbane - South,3GBRI,Greater Brisbane,Toohey,Sunnybank,Moreton
18785,Mackenzie (Brisbane City),NaN,SUB,153.130280,-27.535830,Mackenzie,4156,31000,Brisbane (C),30303106401,...,Rochedale - Burbank,30303,Mt Gravatt,303,Brisbane - South,3GBRI,Greater Brisbane,Mansfield,Mansfield,Bonner
21390,Mount Crosby,NaN,LOCB,152.811951,-27.523056,Mount Crosby,4306,31000,Brisbane (C),31003129010,...,Karana Downs,31003,Ipswich Inner,310,Ipswich,3GBRI,Greater Brisbane,Moggill,Moggill,Blair
21391,Mount Crosby,Mount Belle Vue,MT,152.799053,-27.528131,Mount Crosby,4306,31000,Brisbane (C),31003129010,...,Karana Downs,31003,Ipswich Inner,310,Ipswich,3GBRI,Greater Brisbane,Moggill,Moggill,Blair
21674,Mount Gravatt,NaN,HILL,153.072780,-27.542780,Mount Gravatt,4122,31000,Brisbane (C),30303106324,...,Mount Gravatt,30303,Mt Gravatt,303,Brisbane - South,3GBRI,Greater Brisbane,Mansfield,Mansfield,Bonner
21675,Mount Gravatt,NaN,SUB,153.072220,-27.540280,Mount Gravatt,4122,31000,Brisbane (C),30303106324,...,Mount Gravatt,30303,Mt Gravatt,303,Brisbane - South,3GBRI,Greater Brisbane,Mansfield,Mansfield,Bonner


Remove text in brackets and contract "Mount" to "Mt" so the place names match to the Kerbside localities.
MacGregor will need to be updated to all lower case.

In [13]:
place_concord["Suburbs"] = place_concord["Suburbs"].str.replace("\\(.+\\)", "", regex=True)
place_concord["Suburbs"] = place_concord["Suburbs"].str.replace(r"\bMount\b", "Mt", regex=True)
place_concord.loc[place_concord["Suburbs"] == "MacGregor", "Suburbs"] = "Macgregor"
place_concord["Suburbs"] = place_concord["Suburbs"].str.strip()

Repeat check to confirm that place names were updated correctly

In [14]:
locality_check = kerbside["Suburbs"].isin(place_concord["Suburbs"])
kerbside[~locality_check]

,Suburbs,Collection_Date


Combine kerbside locality information with place concordance locality information to get the SA2 code for each area. SA2 code required for SEIFA decile. Left join used as all suburbs in the kerbside list should be kept for completion.

In [15]:
kerbside_sa2 = kerbside.merge(place_concord, on="Suburbs", how="left")


Confirm that merge was successful

In [16]:
print(kerbside_sa2.head(10))
print(kerbside_sa2.dtypes)
print(kerbside_sa2.shape)

        Suburbs Collection_Date Alternative place name (2020)  \
0  Acacia Ridge       29-Aug-22                           NaN   
1      Algester       11-Jul-22                           NaN   
2     Calamvale       11-Jul-22                           NaN   
3     Parkinson       11-Jul-22                           NaN   
4  Auchenflower       18-Jul-22                           NaN   
5        Milton       18-Jul-22                           NaN   
6      St Lucia       18-Jul-22                           NaN   
7       Taringa       18-Jul-22                           NaN   
8       Anstead       25-Jul-22                           NaN   
9    Bellbowrie       25-Jul-22                           NaN   

  Place type (2020)  Place name longitude (GDA94)  \
0               SUB                    153.026108   
1               SUB                    153.033610   
2               SUB                    153.048060   
3               SUB                    153.029170   
4               SUB

Add the SEIFA deciles to the kerbside dates. Left join used as all suburbs in the kerbside list should be kept for completion.

In [17]:
kerbside_sa2 = kerbside_sa2.merge(seifa_16, on="sa2_16_code", how="left")

print(kerbside_sa2.head(10))
print(kerbside_sa2.dtypes)
print(kerbside_sa2.shape)

        Suburbs Collection_Date Alternative place name (2020)  \
0  Acacia Ridge       29-Aug-22                           NaN   
1      Algester       11-Jul-22                           NaN   
2     Calamvale       11-Jul-22                           NaN   
3     Parkinson       11-Jul-22                           NaN   
4  Auchenflower       18-Jul-22                           NaN   
5        Milton       18-Jul-22                           NaN   
6      St Lucia       18-Jul-22                           NaN   
7       Taringa       18-Jul-22                           NaN   
8       Anstead       25-Jul-22                           NaN   
9    Bellbowrie       25-Jul-22                           NaN   

  Place type (2020)  Place name longitude (GDA94)  \
0               SUB                    153.026108   
1               SUB                    153.033610   
2               SUB                    153.048060   
3               SUB                    153.029170   
4               SUB

Check for any duplicates introduced from the merge

In [18]:
possible_duplicates = kerbside_sa2.duplicated(
    ["sa2_16_code", "Suburbs", "state_rank"]
    , keep = False)
kerbside_sa2.loc[possible_duplicates, ["Suburbs", "Postcode (2019)", "state_rank", "sa2_16_code"]]
# All true duplicates so first or last pattern doesn't matter
kerbside_sa2.drop_duplicates(["sa2_16_code", "Suburbs", "state_rank"], keep="first", inplace=True)

Manually drop the duplicates that don't share the same SA2 name. Using a string distance measure on the suburb against the SA2 name might be a way to improve this.

In [19]:
kerbside_sa2["key"] = kerbside_sa2["Suburbs"] + "~" + kerbside_sa2["sa2_16_code"].astype("str")

kerbside_sa2.loc[kerbside_sa2["Suburbs"].duplicated(keep=False)
    , ["Suburbs", "Postcode (2019)", "state_rank", "sa2_16_name", "key"]]

,Suburbs,Postcode (2019),state_rank,sa2_16_name,key
12,Kholo,4306,461,Karana Downs,Kholo~310031290
13,Kholo,4306,NaN,NaN,Kholo~310021279
16,Moggill,4069,512,Pinjarra Hills - Pullenvale,Moggill~304021091
17,Moggill,4070,479,Bellbowrie - Moggill,Moggill~304021086
42,Oxley,4074,185,Darra - Sumner,Oxley~310011271
43,Oxley,4075,399,Oxley (Qld),Oxley~310011275
91,Nundah,4017,373,Sandgate - Shorncliffe,Nundah~302041044
92,Nundah,4012,379,Nundah,Nundah~302031040
94,Kedron,4032,406,Chermside West,Kedron~302021029
95,Kedron,4031,444,Kedron - Gordon Park,Kedron~302021031


In [20]:
keys_remove = pd.Series(["Kholo~310021279","Moggill~304021091", "Oxley~310011271", "Nundah~302041044" 
, "Kedron~302021029", "Bulimba~303011047", "Tingalpa~301011002", "South Brisbane~305011107"
, "Yeerongpilly~303061078"])
kerbside_sa2 = kerbside_sa2.loc[~(kerbside_sa2["key"].isin(keys_remove))]

### User requirements and export data

Beneficial to sort by date to know what suburbs are coming up

In [21]:
kerbside_sa2["Collection_Date"] = pd.to_datetime(kerbside_sa2["Collection_Date"])
clean_kerb = kerbside_sa2.sort_values("Collection_Date")

Export as csv for quick sharing. In addition to state SA2 SEIFA, have kept the minimum and maximum SA1 scores to reflect the variability in suburbs level of advantage or disadvantage.

In [22]:
clean_kerb = clean_kerb[["Suburbs", "Collection_Date", "state_rank", "state_decile", "state_percentile", "min_sa1_score", "max_sa1_score"]]
clean_kerb.to_csv(
    "../data/kerbside collection with state seifa decile and rank.csv"
    , index=False
)